In [137]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

아달린은 딥러닝의 기초가 되는 핵심 개념입니다. 두 모델은 인공 신경망의 초기 형태로서 현대 신경망 구조의 이해와 개발에 중요합니다. 퍼셉트론은 이진 분류기로 시작해 아달린을 거쳐 더 정교한 학습 알고리즘으로 발전했고, 이는 현재의 다층 신경망과 딥러닝 모델의 토대가 되어 복잡한 패턴 인식과 데이터 처리를 가능하게 했습니다.

In [138]:
iris_set = load_iris()
n_classes = iris_set.target_names.__len__()

[dataset specification](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_iris.html)
## IRIS Dataset
**class:** [setosa, versicolour, virginica]
**Number of Instances**: `150` (50 in each of three classes)
**Number of Attributes**: `4` numeric, predictive attributes and the class
**Attribute Information**: `sepal length`, `sepal width`, `petal length`, `petal width`

In [139]:
class Dataset:
  def __init__(self, dataset, indices, transform=None, encoder=None):
    self.dataset, self.indices = dataset, indices
    self.transform, self.encoder = transform, encoder
  def __getitem__(self, item: int):
    idx = self.indices[item]
    feature, label = self.dataset.data[idx], self.dataset.target[idx]
    if self.transform: feature = self.transform(feature)
    if self.encoder: label = self.encoder(label)
    return feature, label
  def __len__(self): return len(self.indices)

In [140]:
indices = random.sample(range(iris_set.data.__len__()), 100)
encoder = lambda index: np.eye(n_classes)[index]

# init Datasets
support_set = Dataset(iris_set, indices[:50], encoder=encoder)
query_set = Dataset(iris_set, indices[50:], encoder=encoder)

## 아달린
아달린(Adaptive Linear Neuron, Adaptive Linear Element)는 단층의 인공 신경망입니다. 퍼셉트론에 근거하고 있으며 무게, 절편, 총합 함수를 사용합니다.

### 수식화
$$
y = \Sigma^{n}_{j=0}x_jw_j + \theta
$$
- $x$는 입력 벡터입니다.
    - $x_0 = 1$입니다.
- $w$는 가중치 벡터입니다.
    - $w_0 = 0$입니다.
- $n$은 입력의 갯수입니다.
- $\theta$는 상수입니다.
- $y$는 모델의 출력입니다.

In [143]:
class ADALINE:
  def __init__(self, n_inpt, n_ouput):
    self.n_ouput, self.n_inpt = n_ouput, n_inpt
    self.weight = np.zeros(shape=(self.n_inpt, self.n_ouput))
  # __init__

  def forward(self, x): return relu(np.dot(x.T, self.weight))
# ADALINE

In [142]:
def sigmoid(x): return 1 / (1 + np.exp(-1 * x))
def relu(x): return np.maximum(0, x)

### 학습법 정의하기
아달린의 학습법은 경사 하강법의 일종인 최소 제곱법(least mean squares)이 사용됩니다. 가중치가 갱신되기 위한 알고리즘과 그 변수는 다음과 같습니다.
$$
w \leftarrow w + \eta(o-y)x
$$
- $\eta$는 학습률입니다.
- $y$는 모델의 출력입니다.
- $o$는 출력입니다.
- $E = (o-y)^2$는 오류에 대한 제곱입니다.

In [155]:
def GDR(model, lr):
  def _GDR(x, y):
    pred = model.forward(x)
    error = pred - y
    grads = np.dot(x.reshape(1, -1).T, error.reshape(1, -1))
    model.weight -= grads * lr
  return _GDR

In [158]:
progress_bar = tqdm(range(1000))

# init and train a model
model = ADALINE(4, 3)
optimizer = GDR(model, 0.001)
for _ in progress_bar:
  for feature, label in support_set:
    optimizer(feature, label)

100%|██████████| 1000/1000 [00:01<00:00, 591.85it/s]


In [159]:
count, n_samples = 0, len(query_set)
for feature, label in support_set:
  pred = model.forward(feature)
  if np.argmax(pred) == np.argmax(label): count += 1
print(f"accuracy: {count / n_samples:.2f}({count}/{n_samples})")

accuracy: 0.92(46/50)
